# Importing dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
import torch

In [2]:
torch.__version__

'2.0.1+cpu'

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. GPU device:", torch.cuda.get_device_name())
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


##### CUDA is used to help in write codes that can run on GPU, making them highly efficient for the tasks that can be parallelized.

In [6]:
import gym #the openAI gym library
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env #allows us to work in atari environments
import os

A2C - (Advantage Actor-Critic algorithm). It is a policy gradient based algorithm for RL.
VecFrameStack - wraps multiple environments

# Testing the atari environment

In [7]:
pip install gym[atari]

Note: you may need to restart the kernel to use updated packages.


In [8]:
environment_name = 'Breakout-v0'
env = gym.make(environment_name)

In [9]:
env.reset()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [10]:
env.action_space

Discrete(4)

In [11]:
env.observation_space

Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
 

In [12]:
episodes =10
for episode in range(1,episodes+1):
    obs = env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action = env.action_space.sample() 
        obs,reward,done,info = env.step(action)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode,score))
    

C:\Users\91638\anaconda3\lib\site-packages\gym\envs\atari\environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
C:\Users\91638\AppData\Roaming\Python\Python39\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode:1 Score:0.0
Episode:2 Score:5.0
Episode:3 Score:2.0
Episode:4 Score:2.0
Episode:5 Score:0.0
Episode:6 Score:0.0
Episode:7 Score:0.0
Episode:8 Score:0.0
Episode:9 Score:4.0
Episode:10 Score:1.0


In [13]:
env.close()

# Vectorizing the environment

Vectorizing, generally means that it allows you to train the agent faster by training in parallel.

In [14]:
env = make_atari_env('Breakout-v0',n_envs=4,seed=0) 
#the above line creates  four parallel environments,each running an instance of the Breakout game
env = VecFrameStack(env,n_stack=4)

n_envs = specifies how many parallel environments you need
n_stack = specifies how many consecutive frames you want to stack as the agent's observation

This is done to enhance sample efficiency and capture temporal dependencies

   # Training the model

In [15]:
log_path = os.path.join('Training','Logs')
model = A2C('CnnPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


##### CnnPolicy - Convolutional Neural Network, which is very suitable for image based observations

In [27]:
model.learn(total_timesteps = 10000000)

Logging to Training\Logs\A2C_4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 878      |
|    ep_rew_mean        | 16.6     |
| time/                 |          |
|    fps                | 209      |
|    iterations         | 100      |
|    time_elapsed       | 9        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.226   |
|    explained_variance | 0.334    |
|    learning_rate      | 0.0007   |
|    n_updates          | 100232   |
|    policy_loss        | 0.000388 |
|    value_loss         | 1.21     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 908      |
|    ep_rew_mean        | 17.5     |
| time/                 |          |
|    fps                | 213      |
|    iterations         | 200      |
|    time_elapsed       | 18       |
|    total_timesteps    | 4000     |
| train

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 974      |
|    ep_rew_mean        | 19.5     |
| time/                 |          |
|    fps                | 205      |
|    iterations         | 1400     |
|    time_elapsed       | 136      |
|    total_timesteps    | 28000    |
| train/                |          |
|    entropy_loss       | -0.204   |
|    explained_variance | 0.483    |
|    learning_rate      | 0.0007   |
|    n_updates          | 101532   |
|    policy_loss        | 0.0387   |
|    value_loss         | 0.167    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 972      |
|    ep_rew_mean        | 19.3     |
| time/                 |          |
|    fps                | 205      |
|    iterations         | 1500     |
|    time_elapsed       | 145      |
|    total_timesteps    | 30000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 928      |
|    ep_rew_mean        | 17.5     |
| time/                 |          |
|    fps                | 207      |
|    iterations         | 2800     |
|    time_elapsed       | 269      |
|    total_timesteps    | 56000    |
| train/                |          |
|    entropy_loss       | -0.14    |
|    explained_variance | 0.487    |
|    learning_rate      | 0.0007   |
|    n_updates          | 102932   |
|    policy_loss        | 0.0268   |
|    value_loss         | 0.158    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 937      |
|    ep_rew_mean        | 17.9     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 2900     |
|    time_elapsed       | 280      |
|    total_timesteps    | 58000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 925      |
|    ep_rew_mean        | 17.9     |
| time/                 |          |
|    fps                | 201      |
|    iterations         | 4100     |
|    time_elapsed       | 406      |
|    total_timesteps    | 82000    |
| train/                |          |
|    entropy_loss       | -0.129   |
|    explained_variance | -1.35    |
|    learning_rate      | 0.0007   |
|    n_updates          | 104232   |
|    policy_loss        | 0.316    |
|    value_loss         | 0.584    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 926      |
|    ep_rew_mean        | 18.1     |
| time/                 |          |
|    fps                | 201      |
|    iterations         | 4200     |
|    time_elapsed       | 416      |
|    total_timesteps    | 84000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 948      |
|    ep_rew_mean        | 18.2     |
| time/                 |          |
|    fps                | 200      |
|    iterations         | 5400     |
|    time_elapsed       | 539      |
|    total_timesteps    | 108000   |
| train/                |          |
|    entropy_loss       | -0.146   |
|    explained_variance | 0.945    |
|    learning_rate      | 0.0007   |
|    n_updates          | 105532   |
|    policy_loss        | 0.0149   |
|    value_loss         | 0.0583   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 960      |
|    ep_rew_mean        | 18.6     |
| time/                 |          |
|    fps                | 200      |
|    iterations         | 5500     |
|    time_elapsed       | 549      |
|    total_timesteps    | 110000   |
| train/                |          |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.05e+03 |
|    ep_rew_mean        | 22.7     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 494600   |
|    time_elapsed       | 47815    |
|    total_timesteps    | 9892000  |
| train/                |          |
|    entropy_loss       | -0.0812  |
|    explained_variance | 0.37     |
|    learning_rate      | 0.0007   |
|    n_updates          | 594732   |
|    policy_loss        | -0.0189  |
|    value_loss         | 0.246    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.06e+03 |
|    ep_rew_mean        | 23.1     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 494700   |
|    time_elapsed       | 47826    |
|    total_timesteps    | 9894000  |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.05e+03 |
|    ep_rew_mean        | 22.9     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 495900   |
|    time_elapsed       | 47951    |
|    total_timesteps    | 9918000  |
| train/                |          |
|    entropy_loss       | -0.125   |
|    explained_variance | 0.889    |
|    learning_rate      | 0.0007   |
|    n_updates          | 596032   |
|    policy_loss        | 0.0146   |
|    value_loss         | 0.0558   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.06e+03 |
|    ep_rew_mean        | 23       |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 496000   |
|    time_elapsed       | 47961    |
|    total_timesteps    | 9920000  |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.04e+03 |
|    ep_rew_mean        | 22.1     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 497200   |
|    time_elapsed       | 48078    |
|    total_timesteps    | 9944000  |
| train/                |          |
|    entropy_loss       | -0.121   |
|    explained_variance | 0.275    |
|    learning_rate      | 0.0007   |
|    n_updates          | 597332   |
|    policy_loss        | 0.0662   |
|    value_loss         | 1.05     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.04e+03 |
|    ep_rew_mean        | 22       |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 497300   |
|    time_elapsed       | 48088    |
|    total_timesteps    | 9946000  |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 963      |
|    ep_rew_mean        | 19.2     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 498500   |
|    time_elapsed       | 48209    |
|    total_timesteps    | 9970000  |
| train/                |          |
|    entropy_loss       | -0.0754  |
|    explained_variance | 0.333    |
|    learning_rate      | 0.0007   |
|    n_updates          | 598632   |
|    policy_loss        | 0.0348   |
|    value_loss         | 0.267    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 973      |
|    ep_rew_mean        | 19.7     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 498600   |
|    time_elapsed       | 48220    |
|    total_timesteps    | 9972000  |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.04e+03 |
|    ep_rew_mean        | 21.4     |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 499900   |
|    time_elapsed       | 48367    |
|    total_timesteps    | 9998000  |
| train/                |          |
|    entropy_loss       | -0.155   |
|    explained_variance | 0.771    |
|    learning_rate      | 0.0007   |
|    n_updates          | 600032   |
|    policy_loss        | 0.000388 |
|    value_loss         | 0.0276   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.03e+03 |
|    ep_rew_mean        | 21       |
| time/                 |          |
|    fps                | 206      |
|    iterations         | 500000   |
|    time_elapsed       | 48377    |
|    total_timesteps    | 10000000 |
| train/                |          |
|

# Saving the model

In [28]:
a2c_path = os.path.join('Training','Saved Models','1_A2C_Breakout_Model')
model.save(a2c_path)

In [19]:
del model

In [20]:
model = A2C.load(a2c_path,env)

Wrapping the env in a VecTransposeImage.


Deleting and reloading the model is done just to check whether the model was saved correctly.

# Evaluation and Testing

We have trained our model using a vectorized environment with 4 instances parallely training to make the training faster. But it cant be done while evaluating the model. So, we have to bring it back to the original non vectorized environment.

In [29]:
env = make_atari_env('Breakout-v0',n_envs=1,seed=0)
env = VecFrameStack(env,n_stack=4)

In [30]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

(27.2, 10.067770358922575)

In [31]:
env.close()